In [1]:
import openai
from openai import OpenAI

import networkx
from networkx.algorithms.community import k_clique_communities
import networkx.algorithms.approximation as approximation

import numpy
from scipy import stats
import pandas

import matplotlib
import matplotlib.pyplot
import matplotlib.animation as animation
import matplotlib.patches as patches
import seaborn
import plotly.graph_objects as go

import os.path
import datetime
import re
import json
import random

In [3]:
parsed_data_folder = 'parsed-data'
gpt_key = ""
with open("../gpt-key", 'r') as file:
    gpt_key = file.read()
openai_assistant = "asst_vEl67u700hgUlBWHUAw9ApzV"

In [4]:
client = OpenAI(api_key=gpt_key)
thread = client.beta.threads.create()
assistant_content = "You're a helpful mathematics history tutor. Your answers are short and strictly follow instructions. For questions you answer shortly and to the point, preferring lists and few words over elaborate answers."
print(thread)

Thread(id='thread_CQ0H7fceDynRdjpQnzOgfkkf', created_at=1731272939, metadata={}, object='thread', tool_resources=ToolResources(code_interpreter=None, file_search=None))


In [5]:
centuries_list = list(range(-10, 22))

formatted_centuries = [
    f"{abs(c)}{' BC' if c < 0 else ' AD'}" for c in centuries_list
]

In [6]:
list_mathematicians_flag = False # to control for accidental triggering

mathematicians_per_century = {}

if list_mathematicians_flag:
    for century in formatted_centuries:
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": assistant_content},
                {"role": "user", "content": f"List all famous mathematicians of {century} century"}
            ]
        )
        response_message = response.choices[0].message.content.strip()

        mathematicians_per_century[century] = response_message

In [7]:
out_file = open(f"{parsed_data_folder}/mathematicians_per_century.json", "w")
json.dump(mathematicians_per_century, out_file)
out_file.close()

In [8]:
mathematicians_curated_dict = {
    "Pythagoras": "c. 570–495 BC",
    "Euclid": "c. 300 BC",
    "Thales of Miletus": "c. 624–546 BC",
    "Anaximander": "c. 610–546 BC",
    "Anaximenes": "c. 586–526 BC",
    "Zeno of Elea": "c. 490–430 BC",
    "Anaxagoras": "c. 500–428 BC",
    "Archimedes": "c. 287–212 BC",
    "Apollonius of Perga": "c. 262–190 BC",
    "Eratosthenes": "c. 276–194 BC",
    "Hipparchus": "c. 190–120 BC",
    "Ptolemy": "c. 100–170 AD",
    "Heron of Alexandria": "c. 10–70 AD",
    "Menelaus of Alexandria": "c. 70–140 AD",
    "Diophantus": "c. 200–284 AD",
    "Hypatia of Alexandria": "c. 360–415 AD",
    "Hero of Alexandria": "c. 10–70 AD",
    "Simplicius": "c. 490–560 AD",
    "Proclus": "412–485 AD",
    "Aryabhata": "476–550 AD",
    "Brahmagupta": "598–668 AD",
    "Xuanzang": "602–664 AD",
    "Al-Khwarizmi": "c. 780–850 AD",
    "Omar Khayyam": "1048–1131 AD",
    "Thabit ibn Qurra": "826–901 AD",
    "Al-Battani": "c. 858–929 AD",
    "Ibn al-Haytham": "c. 965–1040 AD",
    "Al-Biruni": "973–1048 AD",
    "Fibonacci": "c. 1170–1250 AD",
    "Ibn Sina": "980–1037 AD",
    "Al-Farabi": "c. 872–950 AD",
    "Nasir al-Din al-Tusi": "1201–1274 AD",
    "Wang Xizhi": "303–361 AD",
    "Nicole Oresme": "c. 1320–1382 AD",
    "Johannes Gutenberg": "c. 1400–1468 AD",
    "Madhava of Sangamagrama": "c. 1350–1425 AD",
    "Zhu Shijie": "c. 1260–1320 AD",
    "Nicolaus Copernicus": "1473–1543 AD",
    "Regiomontanus": "1436–1476 AD",
    "Girolamo Cardano": "1501–1576 AD",
    "John Napier": "1550–1617 AD",
    "Johannes Kepler": "1571–1630 AD",
    "Tycho Brahe": "1546–1601 AD",
    "Simon Stevin": "1548–1620 AD",
    "Galileo Galilei": "1564–1642 AD",
    "François Viète": "1540–1603 AD",
    "René Descartes": "1596–1650 AD",
    "Isaac Barrow": "1630–1677 AD",
    "Pierre de Fermat": "1607–1665 AD",
    "Blaise Pascal": "1623–1662 AD",
    "Christiaan Huygens": "1629–1695 AD",
    "John Wallis": "1616–1703 AD",
    "Gottfried Wilhelm Leibniz": "1646–1716 AD",
    "Isaac Newton": "1643–1727 AD",
    "Leonhard Euler": "1707–1783 AD",
    "Joseph-Louis Lagrange": "1736–1813 AD",
    "Pierre-Simon Laplace": "1749–1827 AD",
    "Carl Friedrich Gauss": "1777–1855 AD",
    "Henri Poincaré": "1854–1912 AD",
    "Augustin-Louis Cauchy": "1789–1857 AD",
    "Adrien-Marie Legendre": "1752–1833 AD",
    "Bernhard Riemann": "1826–1866 AD",
    "Évariste Galois": "1811–1832 AD",
    "Niels Henrik Abel": "1802–1829 AD",
    "George Boole": "1815–1864 AD",
    "David Hilbert": "1862–1943 AD",
    "Felix Klein": "1849–1925 AD",
    "Ada Lovelace": "1815–1852 AD",
    "Kurt Gödel": "1906–1978 AD",
    "John von Neumann": "1903–1957 AD",
    "André Weil": "1906–1998 AD",
    "Paul Erdős": "1913–1996 AD",
    "Alexander Grothendieck": "1928–2014 AD",
    "Emmy Noether": "1882–1935 AD",
    "Alan Turing": "1912–1954 AD",
    "John Nash": "1928–2015 AD",
    "Benoît Mandelbrot": "1924–2010 AD",
    "Andrew Wiles": "1953–present",
    "Terence Tao": "1975–present",
    "John Clay": "unknown",
    "Maryam Mirzakhani": "1977–2017 AD",
    "Manjul Bhargava": "1974–present",
    "Cédric Villani": "1973–present",
    "Peter Scholze": "1987–present",
    "Jean-Pierre Serre": "1926–present",
    "Mary Cartwright": "1900–1998 AD",
    "Julia Robinson": "1919–1985 AD",
    "Paul Dirac": "1902–1984 AD",
    "Émile Picard": "1856–1941 AD",
    "Jacques Hadamard": "1865–1963 AD",
    "Avner Ash": "unknown",
    "Srinivasa Ramanujan": "1887–1920 AD",
    "Sophie Germain": "1776–1831 AD",
    "Joseph Fourier": "1768–1830 AD",
    "Jean Baptiste Joseph Fourier": "1768–1830 AD",
    "Norbert Wiener": "1894–1964 AD",
    "Bertrand Russell": "1872–1970 AD",
    "Andrey Kolmogorov": "1903–1987 AD",
    "Alexander L. Polyakov": "1945–present",
    "Grigori Perelman": "1966–present",
    "John Horton Conway": "1937–2020 AD",
    "Yitang Zhang": "1955–present",
    "Ingrid Daubechies": "1954–present"
}

In [9]:
client = OpenAI(api_key=gpt_key)
thread = client.beta.threads.create()
assistant_content = "You're a helpful mathematics history tutor. For a mathematician's name received you list his or her works. You provide only a list of works, nothing more."
print(thread)

Thread(id='thread_vUGJQdu6zLIrmCnXRLATOe1V', created_at=1731274364, metadata={}, object='thread', tool_resources=ToolResources(code_interpreter=None, file_search=None))


In [12]:
list_works_flag = True # to control for accidental triggering

math_works_per_person = {}

if list_works_flag:
    for mathematician in mathematicians_curated_dict.keys():
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": assistant_content},
                {"role": "user", "content": mathematician}
            ]
        )
        response_message = response.choices[0].message.content.strip()

        math_works_per_person[mathematician] = response_message

In [13]:
out_file = open(f"{parsed_data_folder}/math_works_per_person.json", "w")
json.dump(math_works_per_person, out_file)
out_file.close()